<a href="https://colab.research.google.com/github/martapavelka/scpc/blob/dev/scpc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A Software for Detecting Closure Properties in Simplicial Complexes  

This program by <i>Marta Pavelka</i> detects the following simplicial complex closure properties: under-closed, semi-closed, weakly-closed, chordal, closed, and unit-interval. To use it, simply enter a list of facets of a simplicial complex and select the desired property. For the definitions of the above properties see the article <a href="https://arxiv.org/pdf/2101.09243.pdf">Hamiltonian paths, closed complexes, and determinantal facet ideals</a> by Bruno Benedetti, Lisa Seccia and Matteo Varbaro.



## Colab usage
1. Set up default values including the list of facets
2. Run first two code cells (setup and relabelling function definition)
3. Find the desired property cell further down
4. Run that cell and check the result at the bottom of the cell

## Fixes and possible improvements

*   Human readable input matrix format error.
*   Trim input matrix empty newlines.
*   Support multiple (standard) input matrix formats.
*   Load Colab heading and description into web UI.
*   Log usage and errors.
*   Cache results.


## Setup

In [1]:
from optparse import OptionParser
import select
import sys

### 
# Set defaults..
###

def_list_of_facets = [[1,2], [1,3], [2,3], [2,4], [3,4], [3,5],[4,5],[4,6], [5,6], [1,5], [1,6], [2,6]]
properties = ("under-closed", "semi-closed", "weakly-closed", "chordal", "closed", "unit-interval",
              "traceable", "hamiltonian", "weakly-traceable", "weakly-hamiltonian")
default_property = properties[0]
vert_min = 2
vert_max = 30

###
# Process arguments
###

parser = OptionParser()
parser.add_option("-p", "--property", action = "store", dest = "property",
                  type = "choice", choices = properties, default = default_property,
                  help = "Property to analyze. Valid properties are %s. Default property is %s." % (properties, default_property))
parser.add_option("-f") # ignore google colab option
(options, args) = parser.parse_args()

###
# Define helper functions
###

def exception (str, code = 1):
  print(str, file=sys.stderr)
  sys.exit(code)

def containsConsecutiveSet (num_list, to):
  return sorted(set(num_list)) == list(range(1, to + 1))

def loadInputMatrix ():
  try:
    matrix = []
    for line in sys.stdin:
      if not line.strip():
        continue
      matrix.append([int(num) for num in line.split() if num != " "])
    return matrix
  except:
    exception("Invalid input, %s occurred." % (sys.exc_info()[0]), 2)

def run (fn, fn_property):
  if default_property != None and options.property != fn_property:
    return
  labeling = fn(list_of_facets, number_of_vertices)
  if labeling:
    print("is %s with the labeling" % (fn_property))
    print(labeling)
  else:
    print("is NOT %s" % (fn_property))
  print()

###
# Process and validate variables
###

if select.select([sys.stdin], [], [], 0.0)[0]:
  list_of_facets = loadInputMatrix()
try:
  if not list_of_facets:
    raise NameError
except NameError:
  list_of_facets = def_list_of_facets
  default_property = None # allow all in case of colab

if len(list_of_facets) < 2:
  exception("The input matrix must contain at least two rows.")
list_of_facets_flat = [item for sublist in list_of_facets for item in sublist]
number_of_vertices = max(list_of_facets_flat)
if number_of_vertices < vert_min or number_of_vertices > vert_max:
  exception("Number of verticies must be in range %d to %d." % (vert_min, vert_max))
if not containsConsecutiveSet(list_of_facets_flat, number_of_vertices):
  exception("Input matrix does not contain a consecutive set from 1 to n.")
dimension = (len(list_of_facets[1]) - 1)

print("Input simplicial complex")
print(list_of_facets)
print()


Input simplicial complex
[[1, 2], [1, 3], [2, 3], [2, 4], [3, 4], [3, 5], [4, 5], [4, 6], [5, 6], [1, 5], [1, 6], [2, 6]]



## Define common *functions*

In [14]:
import itertools

# Function that relabels vertices and therefore facets according to a given permutation 

def facets_relabeling (facets, perm):
  ## Relabaled facets setting to empty array
  relabeled_facets = []
  ## Number of vertices in a facet (dimension of the complex + 1)
  d = len(facets[1])
  ## Going througt all the facets of the complex and relabeling
  for k in range(0, len(facets)):
    ## Setting the k-th relabaled facet to empy array ready to be filled  
    relabeled_facet = []
    ## Going throuht all the vertices of the k-th facet 
    for l in range(0, d):
      ## Vertex before labeling
      vertex = facets[k][l]
      ## New label of the vertex
      relabaled_vertex = perm[vertex-1]
      ## Adding the new label 
      relabeled_facet.append(relabaled_vertex)
    ## Sorting the labels to get increasing labeling of the facet   
    relabeled_facet.sort()
    ## Adding the relabaled facet to the list of relabaled facets
    relabeled_facets.append(relabeled_facet)

  return relabeled_facets

## Function creating list of all potential facets from the under-closed condition
## Recursively
def expand (input):
  output = []
  for i in range(input[0]+1, input[1]+1):
    if len(input) == 2:
      output.append([input[0], i])
      continue
    input[1] = i
    expanded = expand(input[1:])
    for e in expanded:
      e.insert(0, input[0])
      output.append(e)
  return output

## Function creating list of all potential facets from the second condition
## Recursively

def expand_inv (input):
  inp_len = len(input)
  output = []
  for i in range(input[-1] - 1, input[-2] - 1, -1):
    if len(input) == 2:
      output.append([i, input[1]])
      continue
    input[-2] = i
    expanded = expand_inv(input[:-1])
    for e in expanded:
      e.append(input[-1])
      output.append(e)
  return output

def find_max_index (index, a, curr):
  ans = -1
  index = 0
  for i in range(index, len(a)):
    if a[i] > curr:
      if ans == -1:
        ans = curr
        index = i
      else:
        ans = min(ans, a[i])
        index = i
  return index

## Create power set of a list (set of all subsets). This is a generator.
def power_set(seq):
  if len(seq) <= 1:
    yield seq
    yield []
  else:
    for item in power_set(seq[1:]):
      yield [seq[0]]+item
      yield item


## Under-closed check
A pure $d$-dimensional simplicial complex $\Delta$ is under-closed if there is a vertex labeling of $\Delta$ such that for every $d$-face $F=a_0a_1\dots a_d$ (written with $a_0<a_1<\dots <a_d$) the complex $\Delta$ contains all faces of the form $a_0b_1b_2\dots b_d$ with $b_1\leq a_1$, $b_2\leq a_2$, $\dots$, $b_d\leq a_d$. 

In [3]:
####################################################################

## Checking under-closed condition for a given labeling 
## Returns True if the labeling is under-closed
## Returns False if the labeling is NOT under-closed

def is_underclosed_labeling (facets_labeling):
  ## Going througt all the facets of the CS 
  for k in range(0, len(facets_labeling)):
  ## Generating list of potencial facets G for given F
    under_faces = expand(facets_labeling[k])
    for G in under_faces:
      ## If G not in the original list of facets --> not under-closed; return False
      if not G in facets_labeling:
        #print('The labeling', facets_labeling, 'is NOT under-closed. For example the face', G, 'is missing.')
        return False
  ## If for all facets condition true --> under-closed; return True
  return True

######################################################################

## Check UC for all labelings
## For every permutation on 'number_of_vertices' elements, we check the desired condition.

def underclosed_SC (facets_list, number_of_vert):
  ## Go through all the permutations
  def_perm = list(range(1, number_of_vert + 1))
  for perm in itertools.permutations(def_perm):
    ## Relable the vertices according to new permutation
    new_labeling = facets_relabeling(list_of_facets, perm)
    ## Check if this labeling makes the CS under-closed
    if is_underclosed_labeling(new_labeling):
      return new_labeling
  return []
 
 ########################################################################

run(underclosed_SC, properties[0])


is NOT under-closed



## Semi-closed check

A pure $d$-dimensional simplicial complex $\Delta$ is semi-closed if there is a vertex labeling of $\Delta$ such that for every $d$-face $F=a_0a_1\dots a_d$ (written with $a_0 < a_1<\dots < a_d$) at least one of the following conditions hold:
1. (*underclosed condition*) the complex $\Delta$ contains all faces of the form $a_0b_1b_2\dots b_d$ with $b_1\leq a_1$, $b_2\leq a_2$, $\dots$, $b_d\leq a_d$, or
2. the complex $\Delta$ contains all faces of the form $i_0i_1 \dots i_{d-1}a_d$ with $i_0\geq a_0$, $i_1\geq a_1$, $\dots$, $i_{d-1} \geq a_{d-1}$. 

In [4]:
###########################################################################

## Checking semi-closed condition for a given labeling 
## Returns True if the labeling is semi-closed
## Returns False if the labeling is NOT semi-closed

def is_semi_closed_labeling (facets_labeling):
  ## Going througt all the facets of the CS 
  for k in range(0, len(facets_labeling)):
  ## Generating lists of potencial facets G for given F
    under_faces = expand(facets_labeling[k])
    above_faces = expand_inv(facets_labeling[k])
    ## Set under and semi_two to True.
    ## Will change to False if some face is missing  
    under = True
    semi_two = True
    for G in under_faces:
      ## If G not in the underclosed list of facets --> not underclosed; set under to False
      if not G in facets_labeling:
        under = False
        G_under_false = G
        break
    for G in above_faces:
      ## If G not in the semi-closed list of facets --> not SM; set semi_two to False
      if not G in facets_labeling:
        semi_two = False
        G_semi_false = G
        break
    ## If both conditions for a given facet False, then the labeling NOT SM
    if not under and not semi_two:
      #print('The labeling', facets_labeling, 'is NOT semi-closed. One of the following facets would have to be present:', G_under_false, 'or', G_semi_false, 'for the facet', facets_labeling[k] )
      return False
  ## If at least one of the conditions for all the facets True, then the labeling is SM  
  #print('The labeling', facets_labeling, 'is semi-closed.')
  return True 
#######################################################################

## For every permutation on 'number_of_vertices' elements, we check the desired condition.

def semi_closed_SC(facets_list, number_of_vert):
  ## Go through all the permutations
  def_perm = list(range(1, number_of_vert + 1))
  for perm in itertools.permutations(def_perm):
    ## Relable the vertices according to new permutation
    new_labeling = facets_relabeling(list_of_facets, perm)
    ## Check if this labeling makes the CS underclosed
    if is_semi_closed_labeling(new_labeling):
      return new_labeling
  return []
#########################################################################

run(semi_closed_SC, properties[1])


is semi-closed with the labeling
[[1, 2], [1, 3], [2, 3], [2, 6], [3, 6], [3, 5], [5, 6], [4, 6], [4, 5], [1, 5], [1, 4], [2, 4]]



## Weakly-closed check

A pure $d$-dimensional simplicial complex $\Delta$ is weakly-closed if there is a vertex labeling of $\Delta$ such that for every $d$-face $F=a_0a_1\dots a_d$ (written with $a_0<a_1<\dots <a_d$) and for every $g\notin F$ with $a_0<g<a_d$ there exists a $d$-face $G$ adjacent to $F$ containing $g$ such that either $\max G \neq \max F$ or $\min G \neq \min F$.

In [5]:
## Function to check WC condition for a given facet 
def is_WC_facet (F, facets_labeling):
  ## Find all g such that a_0 < g < a_d
  b = [*range(F[0], F[-1] + 1)]
  ## Filter those in F
  gs = list(set(b) - set(F)) 
  ## For each g with given condition
  for g in gs:
    ## Go through all the facets in the CS containig g and check max/min and adjecency
    g_ok = False
    for G in facets_labeling:
      if g in G and (F[0] != G[0] or F[-1] !=G [-1]) and len(set(F).intersection(G)) == len(F) - 1:
        g_ok = True
        break
    if not g_ok:
      #!!!print('Missing a suitable facet for ', g, 'adjecent to ', F)
      return False
  #print('WC condition ok for', F)
  return True 

#####################################################################
## Check if a given labeling has the weakly closed condition
def is_WC_labeling (facets_labeling): 
  ## Check all facet with the above function
  for F in facets_labeling:
    if not is_WC_facet(F, facets_labeling):
      #print('labeling', facets_labeling, 'is not WC')
      return False
  #print('labeling', facets_labeling, 'is WC')
  return True

###################################################################
## For every permutation on 'number_of_vertices' elements, we check the desired condition.

def WC_SC (facets_list, number_of_vert):
  ## Go through all the permutations
  def_perm = list(range(1, number_of_vert + 1))
  for perm in itertools.permutations(def_perm):
    ## Relable the vertices according to new permutation
    new_labeling = facets_relabeling(list_of_facets, perm)
    ## Check if this labeling makes the CS underclosed
    if is_WC_labeling(new_labeling):
      return new_labeling
  return []

##################################################################
run(WC_SC, properties[2])


is weakly-closed with the labeling
[[1, 2], [1, 3], [2, 3], [2, 4], [3, 4], [3, 5], [4, 5], [4, 6], [5, 6], [1, 5], [1, 6], [2, 6]]



## d-chordal check
A pure $d$-dimensional simplicial complex $\Delta$ is d-chordal if there is a vertex labeling of $\Delta$ such that for every pair of $d$-faces $F=a_0a_1\dots a_d$ and $G=b_0b_1\dots b_d$ (written in increasing order) with  $a_d=b_d$, the complex $\Delta$ contains the whole $d$-skeleton of the simplex on $F \cup G$.

In [6]:
## Function to check d-chordal condition for given pair of facets *with the same maximum*

## A tool to print all combinations of given length 
from itertools import combinations
 
## Note: not checking for the same max of F and G 
def d_chordal_for_pair_of_facets (F, G, facets_labeling):
  ## Do union of elements of F and G without repetitions
  union_F_G = list(set(F + G))
  ## Get all combinations of F union G of lenght len(F) == d-skeleton on F unoin G
  ## Already sorted!
  d_skeleton = [*combinations(union_F_G, len(F))]
  for H in d_skeleton:
    L=[*H]
    if L not in facets_labeling:
      # print('This labeling is not d-chordal. Missing face', L, 'for the pair', F, G)
      return False
  return True 

##############################################################################
## Fucntion to check if a given labeling is d-chordal
## Find all facet pairs with the same maximum and check d-chordality condition for them 
def is_d_chordal_labeling (facets_labeling):
  ## Going through all the facet pairs (F, G)
  for var in combinations(facets_labeling, 2): 
    ## If the maximum of F and G is the same, check the d-chord condition
    if var[0][-1] == var[1][-1]: 
      if not d_chordal_for_pair_of_facets(var[0], var[1], facets_labeling):
        #print('This labeling is not d-chordal. The trouble pair is', var[0], var[1] )
        return False 
      #print('ok pair')
  #print('The labeling', facets_labeling, 'is d-chordal')   
  return True

#############################################################################
## For every permutation on 'number_of_vertices' elements, we check the desired condition.

def d_chordal_SC (facets_list, number_of_vert):
  ## Go through all the permutations
  def_perm = list(range(1, number_of_vert + 1))
  for perm in itertools.permutations(def_perm):
    ## Relable the vertices according to new permutation
    new_labeling = facets_relabeling(list_of_facets, perm)
    ## Check if this labeling makes the CS underclosed
    if is_d_chordal_labeling(new_labeling):
      return new_labeling
  return []

##############################################################################

run(d_chordal_SC, properties[3])


is NOT chordal



## Closed check
 A pure $d$-dimensional simplicial complex $\Delta$ is closed if there exists a vertex labeling of $\Delta$ such that for every pair of $d$-faces $F=a_0a_1\dots a_d$ and $G=b_0b_1\dots b_d$ (written in increasing order) with  $a_i=b_i$ for some $i$, the complex $\Delta$ contains the full $d$-skeleton of the simplex on $F \cup G$.

In [7]:
## Function to check closed condition for a given pair of facets

## A tool to print all combinations of given length 
from itertools import combinations 

## Note: not checking for the same a_i=b_i of F and G 
def is_closed_pair_of_facets (F, G, facets_labeling):
  ## Do union of elements of F and G without repetitions
  union_F_G = list(set(F + G))
  ## Get all combinations of F union G of lenght len(F) == d-skeleton on F unoin G
  ## Already sorted!
  d_skeleton = [*combinations(union_F_G, len(F))]
  for H in d_skeleton:
    L = [*H]
    if L not in facets_labeling:
     # print('This labeling is not closed. Missing face', L, 'for the pair', F, G)
      return False
  return True 

##############################################################################
## Fucntion to check if a given labeling is closed
## Find all facet pairs with the same i-th coordinate and check closed condition for them
def is_closed_labeling (facets_labeling):
  ## Going through all the facet pairs (F, G)
  n = len(facets_labeling[0])
  for var in combinations(facets_labeling, 2): 
    ## If the i-th coordinate of F and G is the same, check the closed condition
    for i in range(0,n):
      if var[0][i] == var[1][i]: 
        if not is_closed_pair_of_facets(var[0], var[1], facets_labeling):
          #print('This labeling', facets_labeling,' is not closed. The trouble pair is', var[0], var[1] )
          return False 
        break
  #print('The labeling', facets_labeling, 'is closed')   
  return True

#############################################################################
## For every permutation on 'number_of_vertices' elements, we check the desired condition.

def closed_SC (facets_list, number_of_vert):
  ## Go through all the permutations
  def_perm = list(range(1, number_of_vert + 1))
  for perm in itertools.permutations(def_perm):
    ## Relable the vertices according to new permutation
    new_labeling = facets_relabeling(list_of_facets, perm)
    ## Check if this labeling makes the CS underclosed
    if is_closed_labeling(new_labeling):
      return new_labeling
  return []

##############################################################################

run(closed_SC, properties[4])


is NOT closed



## Unit-interval CS check
Let $\Delta$ be a pure $d$-dimensional simplicial complex with $n$ vertices. The complex $\Delta$ is called unit-interval (previously almost-closed) if there exists a labeling $1, \ldots, n$ of its vertices such that  for any $d$-face $F=a_0 a_1 \cdots a_d$ of $\Delta$, the complex $\Delta$ contains the whole $d$-skeleton of the simplex with vertex set $\{a_0, a_0 +1, a_0 + 2, \ldots, a_d\}$.

In [13]:
## Function to check unit-interval (previously almost-closed) condition for a given facet
## A tool to print all combinations of given length 
from itertools import combinations

def is_unit_interval_facet (F, facets_labeling):
  ## Do union of elements of F and G without repetitions
  set_for_F = list(range(F[0], F[-1] + 1))
  # print('facet', F, 'set', set_for_F)
  ## Get all combinations of {a_0, a_0+1, ..., a_d} of lenght len(F) = d-skeleton on the set of vertices
  ## Already sorted!
  d_skeleton = [*combinations(set_for_F, len(F))]
  #print(d_skeleton)
  for H in d_skeleton:
    L = [*H]
    if L not in facets_labeling:
      #print('This labeling is not unit-interval. Missing face', L, 'for the facet', F)
      return False
  #print('The facet', F ,'passed the unit-interval test')
  return True 

##############################################################################
## Fucntion to check if a given labeling is unit-interval
def is_unit_interval_labeling (facets_labeling):
  #print(facets_labeling)
  ## Going through all the facets F
  n = len(facets_labeling[0])
  #print('dimension is', n-1)
  for var in facets_labeling: 
    #print(var)
    ## check the unit-interval condition
    if not is_unit_interval_facet(var, facets_labeling):
      #print('This labeling', facets_labeling,' is not unit-interval. The trouble facet is', var)
      return False 
  #print('The labeling', facets_labeling, 'is unit-interval')   
  return True

##############################################################################
## For every permutation on 'number_of_vertices' elements, we check the desired condition.

def almost_closed_SC(facets_list, number_of_vert):
  ## Go through all the permutations
  def_perm = list(range(1, number_of_vert + 1))
  for perm in itertools.permutations(def_perm):
    ## Relable the vertices according to new permutation
    new_labeling = facets_relabeling(list_of_facets, perm)
    ## Check if this labeling makes the CS underclosed
    if is_unit_interval_labeling(new_labeling):
      return new_labeling
  return []

##############################################################################

run(almost_closed_SC, properties[5])


is NOT unit-interval



## Traceable
A pure $d$-dimensional simplicial complex $\Delta$ with n vertices is Traceable if there is a vertex labeling of $\Delta$ such that $\Delta$ contains facets $H_1$, $H_2$, $\dots$, $H_{n-d}$. Where $H_i$ is the facet $(i, i+1, \dots , i+d)$. 

In [9]:
## Checking Traceable 
## Returns True if the labeling is Traceable
## Returns False if the labeling is NOT Traceable

def is_traceable_labeling (facets_labeling):
  ## Create H_1
  Hi = []
  for k in range (1, dimension + 2):
    Hi.append(k)
    #print(Hi)
  if Hi not in facets_labeling:
    #print('The labeling', facets_labeling, 'is NOT Traceable. ', Hi, ' is missing.')
    return False
  ## Going througt all the H_i's and checking if they belong to the labeling 
  for i in range(2, number_of_vertices - dimension +1):
    #print(i)
    ## Create H_i from H_{i-1}
    # using del list[0] to perform removal of the 1st element
    del Hi[0]
    #print(Hi)
    Hi.append(i + dimension)
    #print(Hi)
    if Hi not in facets_labeling:
      #print('The labeling', facets_labeling, 'is NOT Traceable. ', Hi, ' is missing.')
      return False
  ## If all facets H_i are present --> Traceable; return True
  return True

#print(is_traceable_labeling(list_of_facets))
######################################################################

## Check Traceable for all labelings
## For every permutation on 'number_of_vertices' elements, we check the desired condition.

def traceable_SC (facets_list, number_of_vert):
  ## Go through all the permutations
  def_perm = list(range(1, number_of_vert + 1))
  for perm in itertools.permutations(def_perm):
    ## Relable the vertices according to new permutation
    new_labeling = facets_relabeling(list_of_facets, perm)
    ## Check if this labeling makes the CS tr.
    if is_traceable_labeling(new_labeling):
      return new_labeling
  return []

run(traceable_SC, properties[6])

is traceable with the labeling
[[1, 2], [1, 3], [2, 3], [2, 4], [3, 4], [3, 5], [4, 5], [4, 6], [5, 6], [1, 5], [1, 6], [2, 6]]



## Hamiltonian
A pure $d$-dimensional simplicial complex $\Delta$ with n vertices is Traceable if there is a vertex labeling of $\Delta$ such that $\Delta$ contains facets $H_1$, $H_2$, $\dots$, $H_{n}$. Where $H_i$ is the facet $(i , i+1 , \dots , i+d)$ using modulo $n$ for anyting greater then $n$. For instance $H_n = (1,2,3, \dots d, n)$.

In [10]:
## Checking Hamiltonian 
## Returns True if the labeling is Traceable
## Returns False if the labeling is NOT Traceable

def is_hamiltonian_labeling (facets_labeling):
  ## Create H_1
  Hi = []
  for k in range (1, dimension + 2):
    Hi.append(k)
  if Hi not in facets_labeling:
    #print('The labeling', facets_labeling, 'is NOT Traceable. ', Hi, ' is missing.')
    return False
  ## Going througt all the H_i's and checking if they belong to the labeling
  # i from 2 to n-d 
  for i in range(2, number_of_vertices - dimension + 1):
    #print(i)
    ## Create H_i from H_{i-1}
    # using del list[0] to perform removal of the 1st element
    del Hi[0]
    Hi.append(i + dimension)
    if Hi not in facets_labeling:
      return False
  # i from n-d+1 to n
  for i in range(1,dimension + 1):
    del Hi[0]
    Hi.append(i)
    Hi_sort = sorted(Hi)
    if Hi_sort not in facets_labeling:
      return False
  ## If all facets H_i are present --> Hamiltonian; return True
  return True

#print(is_hamiltonian_labeling(list_of_facets))
######################################################################

## Check Hamiltonian for all labelings
## For every permutation on 'number_of_vertices' elements, we check the desired condition.

def hamiltonian_SC (facets_list, number_of_vert):
  ## Go through all the permutations
  def_perm = list(range(1, number_of_vert + 1))
  for perm in itertools.permutations(def_perm):
    ## Relable the vertices according to new permutation
    new_labeling = facets_relabeling(list_of_facets, perm)
    ## Check if this labeling makes the CS under-closed
    if is_hamiltonian_labeling(new_labeling):
      return new_labeling
  return []

run(hamiltonian_SC, properties[7])

is hamiltonian with the labeling
[[1, 2], [1, 3], [2, 3], [2, 4], [3, 4], [3, 5], [4, 5], [4, 6], [5, 6], [1, 5], [1, 6], [2, 6]]



## Weakly-Traceable
A pure $d$-dimensional simplicial complex $\Delta$ with $n$ vertices is wealky-traceable if there is a vertex labeling of $\Delta$ such that $\Delta$ contains a subset $H_{i_1}$, $H_{i_2}$, $\dots$, $H_{i_k}$ of $\{H_1, H_2 \dots , H_{n-d} \}$ that
1. covers all the vertices and 
2. $H_{i_{j}}$ is incident (non-empty intersection) to $H_{i_{j+1}}$ for each $j \in \{1, 2, \dots , k-1\}$.

Here $H_i$ is the facet $(i, i+1, \dots , i+d)$. 

In [11]:
## Checking if a labeling in W-T 
## Returns True if the labeling is 
## Returns False if the labeling is NOT

def is_weakly_traceable_labeling (facets_labeling):
  #print('-----------------')
  ## Create list oh H_i's
  ## Create H_1
  his = []
  hi = []
  for k in range (1, dimension + 2):
    hi.append(k)
  his.append(hi.copy())
  ## Creating all the other H_i's and adding them to the list of H_i's 
  for i in range(2, number_of_vertices - dimension +1):
    ## Create H_i from H_{i-1}
    # using del list[0] to perform removal of the 1st element
    del hi[0]
    hi.append(i + dimension)
    his.append(hi.copy())
  ## Now his contains all the H_i's
  ## Eliminate all H_i's that are NOT in the labaling
  # print(facets_labeling)
  # print(his)
  new_his = [value for value in his if value in facets_labeling]
  #print(new_his)
  ## We need its power set
  ## Lazily iterate over power set
  for his_subset in power_set(new_his):
    if his_subset != []:
      #print(his_subset)
      his_subset_flat = [item for sublist in his_subset for item in sublist]
      # print(his_subset_flat)
      if all(x in his_subset_flat for x in [*range(1, number_of_vertices+1)]):
        length = len(his_subset)
        for j in range(0, length-1):
          if any(k in his_subset[j] for k in his_subset[j+1]):
            if j == length-2:
              #print(his_subset)
              return True
          else: break
  return False
#print(is_weakly_traceable_labeling(list_of_facets))

######################################################################

######################################################################

## Check Weakly Traceable for all labelings
## For every permutation on 'number_of_vertices' elements, we check the desired condition.

def weakly_traceable_SC (facets_list, number_of_vert):
  ## Go through all the permutations
  def_perm = list(range(1, number_of_vert + 1))
  for perm in itertools.permutations(def_perm):
    ## Relable the vertices according to new permutation
    new_labeling = facets_relabeling(list_of_facets, perm)
    ## Check if this labeling makes the CS under-closed
    if is_weakly_traceable_labeling(new_labeling):
      return new_labeling
  return []

run(weakly_traceable_SC, properties[8])

is weakly-traceable with the labeling
[[1, 2], [1, 3], [2, 3], [2, 4], [3, 4], [3, 5], [4, 5], [4, 6], [5, 6], [1, 5], [1, 6], [2, 6]]



## Weakly-Hamitonian
A $d$-dimensional simplicial complex $\Delta$ is weakly-Hamiltonian if it has a labeling such that $\Delta$ contains a subset $H_{i_1}$
, $\dots$, $H_{i_k}$ of $\{H_1, \dots , H_n\}$ that 

1. altogether cover all vertices, 
2. $H_{i_j}$ is incident to $H_{i_{j+1}}$ for each $j \in \{1, \dots , k −1\}$, 
3. $H_{i_k}$ is incident to $H_{i_1}$.

In [12]:
## Checking if a labeling in W-H 
## Returns True if the labeling is 
## Returns False if the labeling is NOT

def is_weakly_hamiltonian_labeling (facets_labeling):
  #print('-----------------')
  ## Create list oh H_i's
  ## Create H_1
  his = []
  hi = []
  for k in range (1, dimension + 2):
    hi.append(k)
  his.append(hi.copy())
  ## Creating the H_i's i=1, 2, ..., n-d and adding them to the list of H_i's 
  for i in range(2, number_of_vertices - dimension +1):
    ## Create H_i from H_{i-1}
    # using del list[0] to perform removal of the 1st element
    del hi[0]
    hi.append(i + dimension)
    his.append(hi.copy())
  ## Creating the H_i's i=n-d+1, ..., n and adding them to the list of H_i's
  for i in range(1, dimension + 1):
    del hi[0]
    hi.append(i)
    hi_sort = sorted(hi)
    his.append(hi_sort.copy())
  ## Now his contains all the H_i's
  ## Eliminate all H_i's that are NOT in the labaling
  # print(facets_labeling)
  # print(his)
  new_his = [value for value in his if value in facets_labeling]
  # print(new_his)
  ## We need its power set
  ## Lazily iterate over power set
  for his_subset in power_set(new_his):
    if his_subset != []:
      # print(his_subset)
      his_subset_flat = [item for sublist in his_subset for item in sublist]
      # print(his_subset_flat)
      if all(x in his_subset_flat for x in [*range(1, number_of_vertices+1)]):
        if any(m in his_subset[0] for m in his_subset[-1]):
          length = len(his_subset)
          for j in range(0, length-1):
            if any(k in his_subset[j] for k in his_subset[j+1]):
              # print(j, his_subset[j], his_subset[j+1])
              if j == length-2:
                #print(his_subset)
                return True
            else: break
  return False
# print(is_weakly_hamiltonian_labeling(list_of_facets))

######################################################################

######################################################################

# Check Weakly-Hamiltonian for all labelings
# For every permutation on 'number_of_vertices' elements, we check the desired condition.

def weakly_hamiltonian_SC (facets_list, number_of_vert):
  ## Go through all the permutations
  def_perm = list(range(1, number_of_vert + 1))
  for perm in itertools.permutations(def_perm):
    ## Relable the vertices according to new permutation
    new_labeling = facets_relabeling(list_of_facets, perm)
    ## Check if this labeling makes the CS under-closed
    if is_weakly_hamiltonian_labeling(new_labeling):
      return new_labeling
  return []

run(weakly_hamiltonian_SC, properties[9])

is weakly-hamiltonian with the labeling
[[1, 2], [1, 3], [2, 3], [2, 4], [3, 4], [3, 5], [4, 5], [4, 6], [5, 6], [1, 5], [1, 6], [2, 6]]

